In [1]:
import numpy as np
import tensorflow as tf
from collections import OrderedDict
import glob

from vae import ccvae
from load_sentinel import *

In [2]:
attempt=60

bs = 8
kp = 0.7
train_size = bs * 10 * 5#5 scenes
valid_size = bs * 2

recording_interval = 100
resample_interval = 5000

n_filters1 = 64

testlogpath = '../vae_logs/test/' + str(attempt)
trainlogpath = '../vae_logs/train/' + str(attempt)
genlogpath = '../vae_logs/gen/' + str(attempt)

In [ ]:
g = tf.Graph()
with g.as_default():
    with tf.Session() as sess:
        nov_cnn = ccvae(mb=False, len_edge=32, outer=64,
                 middle=128, inner=256, innest=512, h_dim=512, latent_dim=256,
                 tb_imgs_to_display=4)
        
        global_step = tf.Variable(0, name="global_step", trainable=False)
        validator = tf.cast(nov_cnn.variational_lower_bound, tf.int32) # alibi
        optimizer = tf.train.AdamOptimizer(1e-4)
        grads_and_vars = optimizer.compute_gradients(-nov_cnn.variational_lower_bound)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        loss_summary = tf.summary.scalar("loss", -nov_cnn.variational_lower_bound)
        
        all_sums_merged = tf.summary.merge_all()
        
        init = tf.global_variables_initializer()        
        sess.run(init)
        
        saver = tf.train.Saver()
        
        train_writer = tf.summary.FileWriter(trainlogpath)
        valid_writer = tf.summary.FileWriter(testlogpath)
        train_writer.add_graph(sess.graph)

        mb_until = bs
        ep = 0
        step = 0
        X_valid_10, _ = sample_some(valid_size)

        while True:
            if (step%resample_interval == 0):
                X_all_10, _ = sample_some(train_size)

            x_batch = X_all_10[mb_until-bs:mb_until]

            _, step, summ, lo = sess.run([train_op,
                                                   global_step,
                                                   all_sums_merged,
                                                   nov_cnn.variational_lower_bound],
                                                  feed_dict={nov_cnn.X_10: x_batch, nov_cnn.keep_prob: kp})        

            if (step%recording_interval == 0):
                train_writer.add_summary(summ, step)  
 #               step, summ, lo = sess.run([global_step,
 #                                           all_sums_merged,
 #                                           nov_cnn.variational_lower_bound],
 #                                          feed_dict={nov_cnn.X_10: X_valid_10, nov_cnn.keep_prob: 1.0})
 #               valid_writer.add_summary(summ, step)    

            mb_until += bs
            if mb_until == train_size:
                mb_until = bs

['/media/ramdisk/S2_10_A.tif', '/media/ramdisk/S2_20_A.tif']
['/media/ramdisk/S2_10_B.tif', '/media/ramdisk/S2_20_B.tif']
['/media/ramdisk/S2_10_C.tif', '/media/ramdisk/S2_20_C.tif']
['/media/ramdisk/S2_10_D.tif', '/media/ramdisk/S2_20_D.tif']
['/media/ramdisk/S2_10_E.tif', '/media/ramdisk/S2_20_E.tif']
['/media/ramdisk/S2_10_A.tif', '/media/ramdisk/S2_20_A.tif']
['/media/ramdisk/S2_10_B.tif', '/media/ramdisk/S2_20_B.tif']
['/media/ramdisk/S2_10_C.tif', '/media/ramdisk/S2_20_C.tif']
['/media/ramdisk/S2_10_D.tif', '/media/ramdisk/S2_20_D.tif']
['/media/ramdisk/S2_10_E.tif', '/media/ramdisk/S2_20_E.tif']
